### Import Libraries

In [5]:
import ccxt
import pandas as pd
import numpy as np
from datetime import date, datetime, timezone, tzinfo
import time, schedule
import matplotlib.pyplot as plt
import ast # Using ast.literal_eval to safely parse the string to a dictionary

In [6]:
import dotenv
import os

# Load variables from the .env file
dotenv.load_dotenv(dotenv.find_dotenv("env"))
os.environ["API_KEY"] = os.getenv("API_KEY")
os.environ["API_SECRET"] = os.getenv("API_SECRET")
os.environ["symbol"] = os.getenv("symbol")
os.environ["pos_size"] = os.getenv("pos_size")
os.environ["params"] = os.getenv("params")
os.environ["target"] = os.getenv("target")

### Connecting to phemex Exchange

In [7]:
phemex = ccxt.phemex({
    'enableRateLimit': True,
    'apiKey': os.environ["API_KEY"],
    'secret': os.environ["API_SECRET"]
})

In [8]:
print(phemex.fetch_balance())

{'info': {'code': '0', 'msg': '', 'data': [{'currency': 'USD', 'balanceEv': '0', 'lockedTradingBalanceEv': '0', 'lockedWithdrawEv': '0', 'lastUpdateTimeNs': '1700182833697790464', 'walletVid': '0'}, {'currency': 'USDT', 'balanceEv': '0', 'lockedTradingBalanceEv': '0', 'lockedWithdrawEv': '0', 'lastUpdateTimeNs': '1700182833637789440', 'walletVid': '0'}]}, 'USD': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'USDT': {'free': 0.0, 'used': 0.0, 'total': 0.0}, 'timestamp': 1700182833697, 'datetime': '2023-11-17T01:00:33.697Z', 'free': {'USD': 0.0, 'USDT': 0.0}, 'used': {'USD': 0.0, 'USDT': 0.0}, 'total': {'USD': 0.0, 'USDT': 0.0}}


### Building the Bot

Strategy:

* Determine the trend with sma20_1d
* Buy/Sell to open around the sma20_15m

### Moving Averages

In [9]:
symbol = os.environ["symbol"] #symbol to trade
pos_size = int(os.environ["pos_size"]) # position size
params = ast.literal_eval(os.environ["params"])
target = int(os.environ["target"])

### Moving Averages

    'timeframes': {                      // empty if the exchange.has['fetchOHLCV'] !== true
        '1m': '1minute',
        '1h': '1hour',
        '1d': '1day',
        '1M': '1month',
        '1y': '1year',
    }

### Determine the Trend

* if bid < sma20_1d then BEARISH
* if bid > sma20_1d then BULLISH

```BUY/SELL TO OPEN AROUND THE SMA20_15m - .1% UNDER AND .3% OVER```

In [10]:
def ask_bid():
    """" ask_bid()[0] == ask"""
    ob = phemex.fetch_order_book(symbol=symbol) # returns the orderbook object wich is a dictionary
    bid = ob['bids'][0][0]
    ask = ob['asks'][0][0]
    return ask, bid

In [11]:
#  FIND DAILY SMA 20
def sma20_1d():

    print('####################### Starting Daily SMA #######################')

    # FETCHING DATA FROM PHEMEX
    timeframe = '1d' # what is the fequency of the bars?
    num_bars = 1000 # how many bars of data will this fetch?
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

    # BUILDING A DATAFRAME FROM THIS DATA
    df_1d = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
    df_1d['timestamp'] = pd.to_datetime(df_1d['timestamp'], unit='ms') # change from milisecond timestamp

    # CALCULATE SMA20d
    df_1d['sma20_1d'] = df_1d['close'].rolling(20).mean()

    # if bid < sma20_1d then BEARISH, if bid > sma20_1d then BULLISH
    bid = ask_bid()[1]
    print(f'##################bid:{bid}####################')
    df_1d.loc[df_1d['sma20_1d']>bid, 'sig'] = 'SELL'
    df_1d.loc[df_1d['sma20_1d']<bid, 'sig'] = 'BUY'
 
    return df_1d

In [12]:
sma20_1d()

####################### Starting Daily SMA #######################
##################bid:2238.12####################


,timestamp,open,high,low,close,volume,sma20_1d,sig
0,2022-06-13,1432.90,1452.70,1161.65,1207.20,130846421.0,NaN,NaN
1,2022-06-14,1207.15,1267.10,1068.90,1206.35,106596975.0,NaN,NaN
2,2022-06-15,1206.35,1250.00,1004.15,1237.05,144729209.0,NaN,NaN
3,2022-06-16,1236.60,1258.40,1050.00,1066.65,186228553.0,NaN,NaN
4,2022-06-17,1066.70,1116.25,1047.90,1084.15,181793108.0,NaN,NaN
...,...,...,...,...,...,...,...,...
549,2023-12-14,2263.63,2333.93,2228.00,2316.77,3629936.0,2197.3100,BUY
550,2023-12-15,2317.55,2319.09,2199.23,2221.83,3771119.0,2204.1975,BUY
551,2023-12-16,2223.40,2263.10,2212.25,2228.12,3163332.0,2212.5250,BUY
552,2023-12-17,2228.10,2246.46,2191.38,2195.81,3246647.0,2220.9415,BUY


Continue with Strategy...

In [13]:
#  FIND 15MINUTE SMA, FIGURE OUT IF BUY OR SELL BASED ON bid vs sma20_1d, if bid < sma20_1d then SELL
def sma20_15m():

    print('####################### Starting 15m SMA #######################')

    # FETCHING DATA FROM PHEMEX
    timeframe = '15m' # what is the fequency of the bars?
    num_bars = 1000 # how many bars of data will this fetch?
    bars = phemex.fetch_ohlcv(symbol=symbol, timeframe=timeframe, limit=num_bars)

    # BUILDING A DATAFRAME FROM THIS DATA
    df_15m = pd.DataFrame(bars, columns=['timestamp','open','high','low','close','volume'])
    df_15m['timestamp'] = pd.to_datetime(df_15m['timestamp'], unit='ms') # change from milisecond timestamp

    # CALCULATE SMA20_15m
    df_15m['sma20_15m'] = df_15m['close'].rolling(20).mean()

    # BUY PRICE 1+2 and SELL PRICE 1+2
    df_15m['bp_1'] = df_15m['sma20_15m'] * 1.001 # sma20_15m 0.1% under and 0.3%over 
    df_15m['bp_2'] = df_15m['sma20_15m'] * 0.997
    df_15m['sp_1'] = df_15m['sma20_15m'] * 0.999
    df_15m['sp_2'] = df_15m['sma20_15m'] * 1.003
    
    return df_15m

In [34]:
sma20_15m()

####################### Starting 15m SMA #######################


,timestamp,open,high,low,close,volume,sma20_15m,bp_1,bp_2,sp_1,sp_2
0,2023-12-08 18:15:00,2354.94,2358.41,2354.56,2355.17,23693.0,NaN,NaN,NaN,NaN,NaN
1,2023-12-08 18:30:00,2355.28,2355.82,2351.77,2352.64,28640.0,NaN,NaN,NaN,NaN,NaN
2,2023-12-08 18:45:00,2352.70,2355.45,2347.87,2348.07,24269.0,NaN,NaN,NaN,NaN,NaN
3,2023-12-08 19:00:00,2347.43,2352.01,2347.27,2351.82,22682.0,NaN,NaN,NaN,NaN,NaN
4,2023-12-08 19:15:00,2353.21,2354.95,2351.44,2353.07,21835.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
995,2023-12-19 03:00:00,2233.90,2235.06,2232.75,2234.08,44523.0,2225.3460,2227.571346,2218.669962,2223.120654,2232.022038
996,2023-12-19 03:15:00,2234.68,2235.04,2232.21,2233.02,25181.0,2226.1425,2228.368642,2219.464072,2223.916357,2232.820927
997,2023-12-19 03:30:00,2232.55,2236.27,2232.55,2235.28,24822.0,2227.2180,2229.445218,2220.536346,2224.990782,2233.899654
998,2023-12-19 03:45:00,2236.09,2238.46,2235.23,2237.06,34391.0,2228.4045,2230.632904,2221.719286,2226.176095,2235.089713


### Code the bot

In [35]:
def open_positions():
    params = {'type':'swap', 'code':'USD'}
    phe_bal = phemex.fetch_balance(params=params)
    open_positions = phe_bal['info']['data']['positions']
    open_positions_side = open_positions[0]['side']
    open_positions_size = open_positions[0]['size']

    if open_positions_side == ('Buy'):
        openpos_bool = True
        long = True
    elif open_positions_side == ('Sell'):
        openpos_bool = True
        long = False
    else:
        openpos_bool = False
        long = None
    return open_positions, openpos_bool, open_positions_size, long

In [ ]:
# CREATE A KILL_SWITCH
def kill_switch():
    print('starting kill switch')
    openposi = open_positions()[1]
    kill_size = open_positions()[2]
    long = open_positions()[3]

    while openposi == True:
        print('starting kill switch loop till limit fill..')
        temp_df = pd.DataFrame()
        print("just made a temporary df")

        phemex.cancel_all_orders(symbol=symbol)
        openposi = open_positions()[1]
        kill_size = open_positions()[2]
        long = open_positions()[3]

        ask = ask_bid()[0]
        bid = ask_bid()[1]

        if long == False:
            phemex.create_limit_buy_order(symbol=symbol, amount=kill_size, price=bid, params=params)
            print(f"just made a BUY order to close order of {kill_size}|{symbol} at ${bid}")
            print('sleeping for 30 seconds to see if it fills')
            time.sleep(30)
        elif long == True:
            phemex.create_limit_sell_order(symbol=symbol, amount=kill_size, price=ask, params=params)
            print(f"just made a BUY order to close order of {kill_size}|{symbol} at ${ask}")
            print('sleeping for 30 seconds to see if it fills')
            time.sleep(30)
        else:
            print("++++++ Something I didn't expect in kill switch function")

        openposi = open_positions()[1]
    return

In [ ]:
kill_switch()

In [ ]:
pos_dict = phemex.fetch_positions(params=params)
pos_dict = pos_dict[0]
pos_dict.keys()

In [ ]:
pos_dict

In [ ]:
def open_position_for_symbol(symbol):
    pos_dict = phemex.fetch_positions(params=params)
    for i in range(len(pos_dict)):
        try:
            if pos_dict[i]['info']['symbol'] == "ETHUSD":
                return pos_dict[i]
        except Exception as exc:
            print('Error fetching open positions')

In [ ]:
# CREATE A PNL CLOSE
def pnl_close():

    current_price = ask_bid()[1]
    # if hit target, close
    print("checking to see if it is time to exit...")
    params = {"type":"swap", "code":"USD"}
    pos_dict = open_position_for_symbol(symbol)
    print(pos_dict)
    side = pos_dict["side"]
    #size = pos_dict["contracts"]
    entry_price = float(pos_dict["entryPrice"])
    leverage = float(pos_dict["leverage"])

    print(f'side:{side} | entry_price:{entry_price} | leverage:{leverage}')

    if side == 'long':
        diff = current_price - entry_price
    else:
        diff = entry_price - current_price
    
    try:
        percentage = round(((diff/entry_price)*leverage), 10)
        print(percentage)
    except:
        percentage = 0
        print(percentage)

    print(f'diff:{diff} | percentage:{percentage}')
    pnl = percentage*100
    #pnl = 26
    
    print(f"this is our pnl {pnl}")

    pnlclose = False
    in_position = False

    if pnl > 0:
        in_position = True
        print("we are in a winning position")
        if pnl > target:
            print(f':) :) :) starting the kill switch becaue we hit our target:{target}')
            pnlclose = True
            kill_switch()
    elif pnl <0:
        print("we are in a loosing position but holding on")
        in_position = True
    else:
        print("we are not in a position")

    return pnlclose, in_position # pnl_close()[0] == pnlclose, pnl_close()[1] == in_position, size 

In [ ]:
pnl_close()

In [ ]:
def bot():
    df_1d = sma20_1d()
    df_15m = sma20_15m()
    ask, bid = ask_bid()
    in_position = pnl_close()[1]

    sig = df_1d.iloc[-1]['sig']
    #sig = 'SELL'
    open_size = pos_size/2

    if in_position == False:
        if sig == "BUY":
            print('making opening order as a BUY')
            bp_1 = df_15m.iloc[-1]['bp_1']
            bp_2 = df_15m.iloc[-1]['bp_2']
            print('bp1:',bp_1)
            print('bp2:',bp_2)

            phemex.cancel_all_orders(symbol=symbol)
            phemex.create_limit_buy_order(symbol=symbol, amount=open_size, price=bp_1, params=params)
            phemex.create_limit_buy_order(symbol=symbol, amount=open_size, price=bp_2, params=params)

            #time.sleep(5)
        else:
            print("making opening order as a SELL")
            sp_1 = df_15m.iloc[-1]['sp_1']
            sp_2 = df_15m.iloc[-1]['sp_2']

            phemex.cancel_all_orders(symbol=symbol)
            phemex.create_limit_sell_order(symbol=symbol, amount=open_size, price=sp_1, params=params)
            phemex.create_limit_sell_order(symbol=symbol, amount=open_size, price=sp_2, params=params)

            #time.sleep(5)
    else:
        print("we are in position already, so not making new orders...")
    return

In [ ]:
bot()